In [1]:
import csv
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import imutils

In [2]:
def get_mask_info(prediction_path, annotation_file_path, mask_pred_path, mask_gt_path):
    images_names = [f for f in os.listdir(prediction_path)]
    pred_mask_list = []
    gt_mask_list = []
    coords_list = []
    coords_count_list = []
    data = pd.read_csv(annotation_file_path)
    img_name, count_coords = np.unique(data['image_name'], return_counts=True)
    for name in images_names:
        print(name)
        pred_mask_list.append(cv2.imread(mask_pred_path + name))
        gt_mask_list.append(cv2.imread(mask_gt_path + name))
        d = data[data['image_name'] == name[:-9]]
        x = np.array(d['x'])
        y = np.array(d['y'])
        coords = np.stack((y, x), axis=1)
        coords_list.append(coords)
        coords_count_list.append(count_coords[img_name == name[:-9]])
    
    return pred_mask_list, gt_mask_list, coords_list, coords_count_list

In [3]:
prediction_path = "dataset/single-channel-images/predictions/full_experiment_single_channel/"
annotation_file_path = "dataset/raw-data/annotations.csv"
mask_pred_path = os.path.join('dataset/single-channel-images/predictions/full_experiment_single_channel/')
mask_gt_path = os.path.join('dataset/single-channel-images/masks/')

pred_mask_list, gt_mask_list, coords_list, coords_count_list = \
        get_mask_info(prediction_path, annotation_file_path, mask_pred_path, mask_gt_path)

RPE1wt_CEP152+GTU88+PCNT_1_003_001_max_C1_mask.png
RPE1wt_CEP63+CETN2+PCNT_1_004_002_max_C3_mask.png
RPE1wt_CEP152+GTU88+PCNT_1_002_001_max_C1_mask.png
RPE1wt_CP110+GTU88+PCNT_2_003_003_max_C1_mask.png
RPE1wt_CEP152+GTU88+PCNT_1_001_001_max_C1_mask.png
RPE1wt_CEP63+CETN2+PCNT_1_002_001_max_C1_mask.png
RPE1wt_CEP152+GTU88+PCNT_1_004_002_max_C3_mask.png
RPE1wt_CEP63+CETN2+PCNT_1_004_004_max_C3_mask.png
RPE1wt_CEP152+GTU88+PCNT_1_000_003_max_C3_mask.png
RPE1wt_CEP63+CETN2+PCNT_1_000_004_max_C2_mask.png
RPE1wt_CP110+GTU88+PCNT_2_004_000_max_C1_mask.png
RPE1wt_CEP63+CETN2+PCNT_1_003_001_max_C2_mask.png
RPE1wt_CP110+GTU88+PCNT_2_003_004_max_C2_mask.png
RPE1wt_CP110+GTU88+PCNT_2_002_003_max_C2_mask.png
RPE1wt_CEP152+GTU88+PCNT_1_002_001_max_C2_mask.png
RPE1wt_CEP152+GTU88+PCNT_1_002_003_max_C1_mask.png
RPE1wt_CEP152+GTU88+PCNT_1_000_000_max_C2_mask.png
RPE1wt_CEP63+CETN2+PCNT_1_004_003_max_C1_mask.png
RPE1wt_CEP152+GTU88+PCNT_1_003_000_max_C3_mask.png
RPE1wt_CP110+GTU88+PCNT_2_002_001_max_C1_

In [4]:
def compute_threshold_mask(mask, trsh):
    image = mask.copy()
    image[image < trsh] = 0
    #image[image >= trsh] = 255
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    #blurred[blurred < trsh] = 0
    #blurred[blurred >= trsh] = 255
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, -10)
    return thresh

In [5]:
def find_center_coordinates(thresh):
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # loop over the contours
    centers_coords = []
    for c in cnts:
        M = cv2.moments(c)
        if M["m00"] != 0:
            cX = int(M["m01"] / M["m00"])
            cY = int(M["m10"] / M["m00"])
            centers_coords.append(np.array((cX, cY)))
        else:
            centers_coords.append(np.array((-1, -1)))
            
    return np.array(centers_coords)

In [6]:
def assign_pred_center_to_gt(pred_coords, gt_coords):
    assignment_list = []
    new_coords_centriol = np.array(pred_coords[pred_coords != [-1, -1]].reshape(-1, 2))
    temp_coords_centriol = new_coords_centriol.copy()
    for i, coord in enumerate(gt_coords):
        vx = coord[0] - temp_coords_centriol[:, 0]
        vy = coord[1] - temp_coords_centriol[:, 1]
        dist = np.linalg.norm(coord - temp_coords_centriol, axis=1)
        if dist.size != 0:
            min_dist = np.argmin(dist)
            if dist[min_dist] < 3:
                temp_coords_centriol[min_dist] = np.array((np.Inf, np.Inf))
                assignment_list.append((i, min_dist))
    return np.array(assignment_list)

In [7]:
def best_threshold_coord_nb(pred_masks, gt_masks, count_coords, gt_coords):
    best_trsh = 0
    scores_track = []
    scores_track.append(0)
    scnd_track = []
    scnd_track.append(0)
    best_score = 0
    total_count = 0
    j = 0
    for trsh in range(0, 255):
        temp_count = 0
        scnd_count = 0
        for i, (pred, gt) in enumerate(zip(pred_masks, gt_masks)):
            if scnd_track[j] != 0:
                print("threshold : {:3}, {:2}/{:2}, percentage detected centriol {:3.3}, precision {:5}"\
                      .format(trsh, i, len(pred_masks), 100 * scores_track[j] / np.sum(count_coords), 100 * scores_track[j] / scnd_track[j]), end='\r')

            thresh = compute_threshold_mask(pred, trsh)
            centers_coords = find_center_coordinates(thresh)
            new_centers_coords = centers_coords[centers_coords != np.array((-1, -1))].reshape(-1, 2)
            assign_center = assign_pred_center_to_gt(centers_coords, gt_coords[i])
            scnd_count += len(new_centers_coords)
            temp_count += len(assign_center)
        
        scores_track.append(temp_count)
        scnd_track.append(scnd_count)
        if temp_count > best_score:
            best_trsh = trsh
            best_score = temp_count
            total_count = temp_count
        j += 1
    return best_trsh, scores_track, scnd_track

In [9]:
best_trsh = 180

In [15]:
clean_mask_list = []
images_names = [f for f in os.listdir(prediction_path)]
f = open('dataset/single-channel-images/predictions_annotation.csv', 'w')
writer = csv.writer(f)
header = ['image_name', 'focus_id', 'x', 'y']
writer.writerow(header)
for name, img in zip(images_names, pred_mask_list):
    thresh = compute_threshold_mask(img, best_trsh)
    center_centriol = find_center_coordinates(thresh)
    center_centriol_re = center_centriol[center_centriol != np.array((-1, -1))].reshape(-1, 2)
    for i, coord in enumerate(center_centriol_re):
        writer.writerow([name[:-4], i, coord[0], coord[1]])
    
    clean_mask_list.append(thresh)

RPE1wt_CEP152+GTU88+PCNT_1_003_001_max_C1_mask.png
